In [12]:
import os

import gmsh
import matplotlib.pyplot as plt
import meshio
import numpy as np
import pandas as pd

import commons, geometry, utils

In [28]:
Lx = 470
Ly = 470
Lz = 60
Rp = 6
eps_se = 0.5
eps_am = 1 - eps_se
markers = commons.SurfaceMarkers()
CELL_TYPES = commons.CellTypes()
scale = (800/470) * 1e-6
scale_factor = [scale, scale, scale]
outdir = "mesh/laminate/"
utils.make_dir_if_missing(outdir)
msh_path = os.path.join(outdir, 'laminate.msh')
tetr_path = os.path.join(outdir, 'tetr.xdmf')
tria_path = os.path.join(outdir, 'tria.xdmf')

## Build Laminate Geometry of Cylindrical SE Strands

In [3]:
df = 470 * pd.read_csv('data/laminate.csv')
df.head()

,x,y,z
0,160.839113,449.497240,NaN
1,8.388525,73.296053,NaN
2,363.830211,436.152462,NaN
3,352.934738,356.118427,NaN
4,89.346537,267.904715,NaN


In [40]:
gmsh.initialize()
gmsh.model.add('laminate')
# box_se = gmsh.model.occ.addBox(0, 0, 0, Lx, Ly, Lz)
box_am = gmsh.model.occ.addBox(0, 0, Lz, Lx, Ly, Rp)
cylinders = []
for idx in range(df.shape[0]):
    x, y, _ = df.loc[idx, :]
    if (x + Rp >= 470) or (y + Rp >= 470):
        continue
    cyl = gmsh.model.occ.addCylinder(x, y, 0, 0, 0, Lz, Rp)
    cylinders.append(cyl)
    gmsh.model.occ.synchronize()

union = gmsh.model.occ.fuse([(3, box_am)], [(3, c) for c in cylinders])
gmsh.model.occ.synchronize()
phys = gmsh.model.addPhysicalGroup(3, [v for _,v in union[0]], 2)
gmsh.model.occ.synchronize()
right = []
left = []
insulated = []
interface = []
for surf in gmsh.model.occ.getEntities(2):
    com = gmsh.model.occ.getCenterOfMass(surf[0], surf[1])
    if np.isclose(com[2], Lz + Rp):
        right.append(surf[1])
    elif np.isclose(com[2], Lz + 0.5 * Rp):
        if np.isclose(com[1], 235) or np.isclose(com[0], 235):
            print("iko")
            insulated.append(surf[1])
        else:
            interface.append(surf[1])
    elif np.isclose(com[2], 0):
        left.append(surf[1])
    else:
        interface.append(surf[1])
gmsh.model.addPhysicalGroup(2, left, markers.left_cc)
gmsh.model.addPhysicalGroup(2, right, markers.right_cc)
gmsh.model.addPhysicalGroup(2, insulated, markers.insulated)
gmsh.model.addPhysicalGroup(2, interface, markers.am_se_interface)
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write(msh_path)
gmsh.finalize()

iko
iko
iko
iko
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 16 (Circle)
Info    : [ 10%] Meshing curve 17 (Circle)
Info    : [ 10%] Meshing curve 18 (Circle)
Info    : [ 10%] Meshing curve 19 (Circle)
Info    : [ 10%] Meshing curve 20 (Line)
Info    : [ 10%] Meshing curve 21 (Circle)
Info    : [ 10%] Meshing curve 22 (Circle)
Info    : [ 10%] Meshing curve 23 (Circle)
Info    : [ 

Info    : [ 10%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 7 (Plane, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 8 (Plane, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 9 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 9 (Cylinder, MeshAdapt)
Info    : [ 10%] Meshing surface 10 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 10 (Cylinder, MeshAdapt)
Info    : [ 10%] Meshing surface 11 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 12 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 13 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 14 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 15 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 16 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 17 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 18 (Cylinder, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 19 (Cylinder, Fronta

Info    : Done tetrahedrizing 11895 nodes (Wall 0.106579s, CPU 0.106037s)
Info    : Reconstructing mesh...
Info    :  - Creating surface mesh
Info    :  - Identifying boundary edges
Info    :  - Recovering boundary
Info    : Done reconstructing mesh (Wall 0.284147s, CPU 0.276544s)
Info    : Found volume 1
Info    : It. 0 - 0 nodes created - worst tet radius 11.9032 (nodes removed 0 0)
Info    : It. 500 - 0 nodes created - worst tet radius 5.02774 (nodes removed 0 500)
Info    : It. 1000 - 0 nodes created - worst tet radius 4.68077 (nodes removed 0 1000)
Info    : It. 1500 - 0 nodes created - worst tet radius 4.43944 (nodes removed 0 1500)
Info    : It. 2000 - 0 nodes created - worst tet radius 4.22829 (nodes removed 0 2000)
Info    : It. 2500 - 0 nodes created - worst tet radius 4.03921 (nodes removed 0 2500)
Info    : It. 3000 - 0 nodes created - worst tet radius 3.85377 (nodes removed 0 3000)
Info    : It. 3500 - 0 nodes created - worst tet radius 3.68799 (nodes removed 0 3500)
Info 

In [41]:
msh = meshio.read(msh_path)
tetr_unscaled = geometry.create_mesh(msh, CELL_TYPES.tetra)
tetr_unscaled.write(tetr_path)
tetr_scaled = geometry.scale_mesh(tetr_unscaled, CELL_TYPES.tetra, scale_factor=scale_factor)
tetr_scaled.write(tetr_path)
tria_unscaled = geometry.create_mesh(msh, CELL_TYPES.triangle)
tria_unscaled.write(tria_path)
tria_scaled = geometry.scale_mesh(tria_unscaled, CELL_TYPES.triangle, scale_factor=scale_factor)
tria_scaled.write(tria_path)